In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Rise Networks\\Desktop\\End-to-End-Machine-Learning-Project-with-MLflow\\research'

In [3]:
os.chdir("../")

In [11]:
%pwd

'c:\\Users\\Rise Networks\\Desktop\\End-to-End-Machine-Learning-Project-with-MLflow'

In [5]:
# Update the config entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float        # parameters for ElasticNet
    l1_ratio: float     # parameters for ElasticNet
    target_column: str  # parameters for ElasticNet

In [6]:
# update the configuration manager in src config
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_training_config = ModelTrainingConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name= config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name,
        )

        return model_training_config

In [7]:
# updating the components
import os
import pandas as pd
from sklearn.linear_model import ElasticNet
from src.mlProject import logger
import joblib

class ModelTraining:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def train_model(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1)
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]]
        y_test = test_data[[self.config.target_column]]

        # Creating the linear regression model using ElasticNet
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(X_train, y_train)

        # Saving the model using joblib
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))


In [10]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_training_config = ModelTraining(config=model_training_config)
    model_training_config.train_model()
except Exception as e:
    raise e

[2025-03-12 10:24:02,046: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-12 10:24:02,050: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-12 10:24:02,057: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-12 10:24:02,060: INFO: common: Created directory at path: artifacts]
[2025-03-12 10:24:02,063: INFO: common: Created directory at path: artifacts/model_training]
